In [1]:
import pandas as pd
import csv
import sys
import re
import scipy
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from time import time

csv.field_size_limit(sys.maxsize)

131072

In [2]:
df = pd.read_pickle('../data/instances.pkl')
labels = list(set(df['target'].values))

In [3]:
X = []
Y = []

print("Preparing lists...")
for index, row in df.iterrows():
    X.append(row["source_code"])
    Y.append(row["target"])

Preparing lists...


In [4]:
print("Extracting features...")
cv = CountVectorizer(binary=True)
cv.fit(X)
instances = cv.transform(X)

Extracting features...


In [5]:
X_train, X_test, y_train, y_test = train_test_split(instances, Y, train_size = 0.75, random_state=42)

/Users/erickmaziero/virtualenvs/refact_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Default parameters

In [6]:
rf_classifier = RandomForestClassifier(random_state=42, verbose=1, n_jobs=-1)
rf_classifier.fit(X_train, y_train)

/Users/erickmaziero/virtualenvs/refact_env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=42, verbose=1, warm_start=False)

In [7]:
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, rf_classifier.predict(X_test)))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


============ EVALUATION on test set:
0.6678587393831024


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished


# Hyperparametrization

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

rf_classifier2 = RandomForestClassifier(random_state=42)

n_iter_search = 20
random_search = RandomizedSearchCV(rf_classifier2,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=5,
                                   n_jobs=-1)

start = time()
print("Hyperparameter tuning...")
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
  " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, random_search.best_estimator_.predict(X_test)))

Hyperparameter tuning...
